	
Soit $n$ un entier naturel non nul. Une *permutation* $\sigma$ de l'ensemble $⟦ 0, n-1 ⟧$ des entiers compris entre $0$ et $n-1$ est une bijection de cet ensemble dans lui-même.

# Représentation d'une permutation

Nous représenterons une permutation $\sigma$ de $⟦ 0, n-1 ⟧$ par un tableau d'entiers $t$ de longueur $n$, tel que pour tout $i \in ⟦ 0, n-1 ⟧$, l'entier $t.(i)$ soit égal à $\sigma(i)$.

#### Question 1
À quelle condition un tableau $t$ de longueur $n$ représente-t-il une permutation de $⟦ 0, n-1 ⟧$ ?

> Un tableau $t$ de longueur $n$ représente une permutation de $⟦ 0, n-1 ⟧$ si et seulement s'il contient une seule fois chaque entier entre 0 et $n-1$.

En déduire une fonction `est_permutation` qui vérifie si un tableau d'entiers représente une permutation.

In [1]:
let est_permutation tab =
  let n = Array.length tab in
  let valeurs = Array.make n false in
  let i = ref 0 in
  let ok = ref true in
  while !ok && !i < n do
    let k = tab.(!i) in
    if k < 0 || k >= n || valeurs.(k)
    then ok := false
    else      
      begin
        valeurs.(k) <- true ;
        incr i
      end
  done;
  !ok
;;

val est_permutation : int array -> bool = <fun>


#### Question 2
Le support d'une permutation $\sigma$ est l'ensemble des points non fixes de $\sigma$, c'est-à-dire l'ensemble des entiers $i$ tels que $\sigma(i) \neq i$.

Écrire une fonction `support : int array -> int list` qui prend en argument un tableau représentant une permutation et qui renvoie son support.

In [2]:
let support tab =
  let s = ref [] in
  let n = Array.length tab in
  for i = n-1 downto 0 do
    if tab.(i) <> i
    then s := i::!s
  done;
  !s
;;

val support : int array -> int list = <fun>


#### Question 3
La composée de deux permutations $\sigma$ et $\sigma'$ de $⟦ 0, n-1 ⟧$ est leur composée $\sigma \circ \sigma'$ au sens habituel des applications. Il s'agit d'une permutation de $⟦ 0, n-1 ⟧$.

Écrire une fonction `compose` qui prend en argument deux tableaux représentant des permutations et qui renvoie le tableau représentant leur composée.

In [3]:
let compose t1 t2 =
  let n = Array.length t1 in
  if n = 0
  then [||]
  else
    begin
      let t = Array.make n t1.(0) in
      for i = 0 to n-1 do
        t.(i)
        <- t1.(t2.(i))
      done;
      t
    end
;;

val compose : 'a array -> int array -> 'a array = <fun>


# Énumération des permutations
## Ordre sur les permutations
On ordonne les permutations de $⟦ 0, n-1 ⟧$ selon l'ordre lexicographique. Nous allons \'ecrire une fonction qui \'etant donn\'ee une permutation représentée par un tableau $t$ calcule son successeur immédiat. Pour cela, on applique l'algorithme suivant :

* On détermine le plus grand entier $i$ tel que $t.(i) < t.(i+1)$ ;
* On permute $t.(i)$ avec le plus petit élément qui lui soit plus grand parmi $t.(i+1)$,..., $t.(n-1)$ ;
* On trie la portion du tableau située à partir de l'indice $i+1$ compris.

#### Question 4
Écrire une fonction `indice_max_croit` qui prend en argument un tableau $t$ et renvoie le plus grand indice $i$ tel que $t.(i) < t.(i+1)$.

In [4]:
exception TableauDecroissant;;

let indice_max_croit t =
  let n = Array.length t in
  let i = ref (n-2) in
  while !i >= 0 && t.(!i) >= t.(!i+1) do
    decr i
         (* équivalent à i := !i - 1 *)
  done;
  if !i >=0
  then !i
  else raise TableauDecroissant
;;

exception TableauDecroissant


val indice_max_croit : 'a array -> int = <fun>


#### Question 5
Écrire une fonction `permute_max` qui prend en argument un tableau $t$ et un entier $i$ et qui permute dans le tableau $t$ l'élément d'indice $i$ avec le plus petit élément qui lui soit plus grand parmi $t.(i+1)$,..., $t.(n-1)$ (ou $n$ est la longueur de $t$).

In [5]:
(* La fonction suivante réalise l'échange en place des éléments d'indices i et j du tableau tab*)
let echange tab i j =
  let tmp = tab.(i) in
  tab.(i) <- tab.(j);
  tab.(j) <- tmp
;;

let permute_max t i =
  let n = Array.length t in
  let j = ref i in
  for k = i+1 to n-1 do
    if t.(k) > t.(i) && (t.(k) < t.(!j) || !j = i)
    then j := k
  done;
  echange t i !j
;;

val echange : 'a array -> int -> int -> unit = <fun>


val permute_max : 'a array -> int -> unit = <fun>


#### Question 6
Écrire une fonction `trie_droite` qui prend en argument un tableau $t$ et un entier $k$ et qui trie en place la portion du tableau située à partir de l'indice $k$.

In [6]:
(* La fonction suivante renvoie l'indice de l'élément minimal à partir de l'indice i inclus*)
let rec indice_min t i =
  let n = Array.length t in
  if i = n-1
  then n-1
  else let j = indice_min t (i+1) in
       if t.(j) < t.(i)
       then j
       else i
;;


let rec trie_droite t k =
  let n = Array.length t in
  if k < n
  then
    begin
      let i = indice_min t k in
      echange t i k;
      trie_droite t (k+1)
    end
;;

val indice_min : 'a array -> int -> int = <fun>


val trie_droite : 'a array -> int -> unit = <fun>


#### Question 7
En déduire une fonction `suivant` qui prend en argument un tableau $t$ représentant une permutation et modifie le tableau $t$ en place de manière à obtenir la permutation suivante.

In [7]:
let suivant t =
  let i = indice_max_croit t in
  permute_max t i;
  trie_droite t (i+1)
;;

val suivant : 'a array -> unit = <fun>


## Affichage des permutations
#### Question  8
Écrire une fonction `print_perm` prenant en argument une permutation et l'affiche sous la forme de votre choix.

In [8]:
let print_perm t =
  let n = Array.length t in
  for i = 0 to n-1 do
    print_int t.(i);
    print_char ' '
  done;
  print_newline ()
;;

val print_perm : int array -> unit = <fun>


#### Question 9
En déduire une fonction `enumere` qui prend en argument un entier $n$ et qui affiche toutes les permutations de $⟦ 0, n-1 ⟧$. Combien y en a-t-il ?

In [9]:
let enumere n =
  let tab = Array.make n 0 in
  (* On démarre avec 0 1 2 3 ... n-1 *)
  for i = 1 to n-1 do
    tab.(i) <- i
  done ;
  print_perm tab;
  try
    while true do
      suivant tab;
      print_perm tab
    done
  with
  | TableauDecroissant -> print_string("Fini !") ;  print_newline()
;;

val enumere : int -> unit = <fun>


In [10]:
enumere 4;;

0 1 2 3 
0 1 3 2 
0 2 1 3 
0 2 3 1 
0 3 1 2 
0 3 2 1 
1 0 2 3 
1 0 3 2 
1 2 0 3 
1 2 3 0 
1 3 0 2 
1 3 2 0 
2 0 1 3 
2 0 3 1 
2 1 0 3 
2 1 3 0 
2 3 0 1 
2 3 1 0 
3 0 1 2 
3 0 2 1 
3 1 0 2 
3 1 2 0 
3 2 0 1 
3 2 1 0 
Fini !


- : unit = ()


# Pour les plus rapides - Cycles
On appelle *cycle* une permutation $\sigma$ tel qu'il existe $k$ éléments distincts $x_0, \dots,x_{k-1}$ tels que le support de $\sigma$ est $\lbrace x_0, \dots, x_{k-1} \rbrace$, et tels que $\sigma(x_0) = x_1$, $\sigma(x_1) = x_2$, $ \dots$, $\sigma(x_{k-2}) = x_{k-1}$ et $\sigma(x_{k-1}) = x_0$.

Un tel cycle peut être représenté par la liste $[x_0; \dots; x_{k-1}]$.

On peut démontrer que toute permutation se décompose de manière unique comme un produit (commutatif) de cycles de supports disjoints. On représentera un produit de cycles sous la forme d'une liste de listes d'entiers.

#### Question 10
Décomposer (à la main) la permutation `[|4; 6; 7; 1; 2; 5; 3; 0|]` en un produit de cycles disjoints.

> Cette permutation est la composition des cycles `[0; 4; 2; 7]` et `[1; 6; 3]`. 

#### Question 11
Écrire une fonction `compose_cycle` prenant en argument un cycle $c$ et un tableau $t$ représentant une permutation pour laquelle on suppose que les éléments du cycle sont des points fixes, et qui modifie en place le tableau $t$ de sorte que le tableau obtenu représente la composée du cycle et de la permutation initiale.

In [11]:
let compose_cycle c t =
  match c with
  | [] -> ()
  | a::q ->
     let sigma_a = t.(a) in
     let rec applique lst =
       match lst with
       | [] -> () 
       | [b] -> t.(b) <- sigma_a
       | b::d::q -> t.(b) <- t.(d) ; applique (d::q)
     in applique c
;;

val compose_cycle : int list -> 'a array -> unit = <fun>


#### Question 12
En déduire une fonction `produit_cycles` qui prend pour argument un produit de cycles et l'entier $n$ et qui renvoie la permutation de $⟦ 0, n-1 ⟧$ représentée par le produit de cycles.

In [12]:
let produit_cycles lst n =
  let tab = Array.make n 0 in
  for i = 1 to n-1 do
    tab.(i) <- i
  done;
  let rec applique_cycles lc =
    match lc with
    | [] -> ()
    | c::q -> applique_cycles q ; compose_cycle c tab
  in
  applique_cycles lst;
  tab
;;

val produit_cycles : int list list -> int -> int array = <fun>


In [13]:

let q11 = [[0; 4; 2; 7]; [1; 6; 3]];;
produit_cycles q11 8;;

val q11 : int list list = [[0; 4; 2; 7]; [1; 6; 3]]


- : int array = [|4; 6; 7; 1; 2; 5; 3; 0|]


#### Question 13
On souhaite maintenant procéder à l'opération inverse.Écrire une fonction `cycles` qui prend en argument une permutation et qui décompose cette permutation en un produit de cycles de supports disjoints.

In [14]:
let cycles tab =
  let n = Array.length tab in
  (* Le tableau vus permet de marquer les éléments qui sont déjà dans un cycle *)
  let vus = Array.make n false in
  (*La fonction trouve_cycle permet de construire le cycle commençant par l'élément debut *)
  let rec trouve_cycle debut courant  =
    vus.(courant) <- true ;
    let suivant = tab.(courant) in
    if suivant = debut
    then [courant]
    else courant::(trouve_cycle debut suivant)
  in
  let produit = ref [] in
  (* On construit le cycle auquel appartient chaque élément qui n'est pas un point fixe et qui n'a pas encore été vu *)
  for i = 0 to n-1 do
    if tab.(i) <> i && not vus.(i)
    then produit := (trouve_cycle i i)::!produit
  done;
  !produit
;;

val cycles : int array -> int list list = <fun>


In [15]:
let q13 = [|4; 6; 7; 1; 2; 5; 3; 0|];;
cycles q13;;

val q13 : int array = [|4; 6; 7; 1; 2; 5; 3; 0|]


- : int list list = [[1; 6; 3]; [0; 4; 2; 7]]
